# Testinnsening av upersonlig skattemelding med næringspesifikasjon
Denne demoen er ment for å vise hvordan flyten for et sluttbrukersystem kan hente et utkast, gjøre endringer, validere/kontrollere det mot Skatteetatens apier, for å sende det inn via Altinn3. 

In [1]:
try: 
    from altinn3 import *
    from skatteetaten_api import *
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
    from pathlib import Path
except ImportError as e:
    print("Mangler en avhengighet, installer dem via pip")
    !pip install python-jose
    !pip install xmltodict
    !pip install pathlib
    import xmltodict
    from skatteetaten_api import *


In [2]:
idporten_header = get_access_token()

https://oidc-ver2.difi.no/idporten-oidc-provider/authorize?scope=skatteetaten%3Aformueinntekt%2Fskattemelding%20openid&acr_values=Level3&client_id=8d7adad7-b497-40d0-8897-9a9d86c95306&redirect_uri=http%3A%2F%2Flocalhost%3A12345%2Ftoken&response_type=code&state=nEsBCBoc2vRyEkBiKqqR7Q&nonce=1664733706098873&code_challenge=j0ayXX7a2DWXz0-DTbsnPsnv9-a38Kb96Na-vrgrVEw=&code_challenge_method=S256&ui_locales=nb


127.0.0.1 - - [02/Oct/2022 20:01:53] "GET /token?code=_J-mo12vHc0yCveBHHHv5O5VVbrUra8vOwlgGGsFsVc&state=nEsBCBoc2vRyEkBiKqqR7Q HTTP/1.1" 200 -


Authorization token received
{'code': ['_J-mo12vHc0yCveBHHHv5O5VVbrUra8vOwlgGGsFsVc'], 'state': ['nEsBCBoc2vRyEkBiKqqR7Q']}
JS : 
{'access_token': 'eyJraWQiOiJ2UHBaZW9HOGRkTHpmdHMxLWxnc3VnOHNyYVd3bW04dHhJaGJ3Y1h3R01JIiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJWUVVQY2tqRjZHeTdOZlNWWGdRMm1aN2FhbzBieDUxSVlKc2xEaGROMmI4PSIsImlzcyI6Imh0dHBzOlwvXC9vaWRjLXZlcjIuZGlmaS5ub1wvaWRwb3J0ZW4tb2lkYy1wcm92aWRlclwvIiwiY2xpZW50X2FtciI6Im5vbmUiLCJwaWQiOiIxMjg3NzU5NzkyOCIsInRva2VuX3R5cGUiOiJCZWFyZXIiLCJjbGllbnRfaWQiOiI4ZDdhZGFkNy1iNDk3LTQwZDAtODg5Ny05YTlkODZjOTUzMDYiLCJhY3IiOiJMZXZlbDQiLCJzY29wZSI6Im9wZW5pZCBza2F0dGVldGF0ZW46Zm9ybXVlaW5udGVrdFwvc2thdHRlbWVsZGluZyIsImV4cCI6MTY2NDgyMDExNCwiaWF0IjoxNjY0NzMzNzE0LCJjbGllbnRfb3Jnbm8iOiI5NzQ3NjEwNzYiLCJqdGkiOiJkbnM3cUJMVWc2UlFVMzZhd2gtQkVvOEdSNGVvVElFREZEY1RzQ2ZFQjJvIiwiY29uc3VtZXIiOnsiYXV0aG9yaXR5IjoiaXNvNjUyMy1hY3RvcmlkLXVwaXMiLCJJRCI6IjAxOTI6OTc0NzYxMDc2In19.v95ODP52icvWoN29loHMVVzJJ22Cdnwj2fttvXXDZUw346PWH_Y62SrgtdDRoBQnpnCnfcIJFRVrSUGslg_dFmxWzqOanwpX5JzFJr4EYW8akvo_

# Hent utkast og gjeldende
Her legger vi inn fødselsnummeret vi logget oss inn med, Dersom du velger et annet fødselsnummer så må den du logget på med ha tilgang til skattemeldingen du ønsker å hente

#### Parten nedenfor er brukt for demostrasjon, pass på bruk deres egne testparter når dere tester


01014701377 er daglig leder for 811422762 

In [3]:
s = requests.Session()
s.headers = dict(idporten_header)
fnr="12877597928" #oppdater med test fødselsnummerene du har fått tildelt
orgnr_as = "312787016"
innteksår = "2022"

### Utkast

In [4]:
url_utkast = f'https://idporten-api-sbstest.sits.no/api/skattemelding/v2/utkast/{innteksår}/{orgnr_as}'

r = s.get(url_utkast)
r

<Response [200]>

### Gjeldende

In [5]:
url_gjeldende = f'https://idporten-api-sbstest.sits.no/api/skattemelding/v2/{innteksår}/{orgnr_as}'
r_gjeldende = s.get(url_gjeldende)
r_gjeldende

<Response [200]>

## Fastsatt
Her får en _http 404_ om vedkommende ikke har noen fastsetting, rekjørt denne etter du har sendt inn og fått tilbakemdling i Altinn at den har blitt behandlet, du skal nå ha en fastsatt skattemelding om den har blitt sent inn som Komplett

In [6]:
url_fastsatt = f'https://idporten-api-sbstest.sits.no/api/skattemelding/v2/fastsatt/{innteksår}/{orgnr_as}'
r_fastsatt = s.get(url_fastsatt)
r_fastsatt

<Response [404]>

In [7]:
r_fastsatt.headers

{'korrelasjonsid': '6179fbd1-ff70-6209-3e28-8dfdcc21d83c', 'content-type': 'text/plain;charset=UTF-8', 'content-length': '26', 'date': 'Sun, 02 Oct 2022 18:02:21 GMT', 'Expires': 'Thu, 01 Dec 1994 16:00:00 GMT', 'Strict-Transport-Security': 'max-age=16070400; includeSubDomains'}

## Svar fra hent gjeldende 

### Gjeldende dokument referanse: 
I responsen på alle api kallene, være seg utkast/fastsatt eller gjeldene, så følger det med en dokumentreferanse. 
For å kalle valider tjenesten, er en avhengig av å bruke korrekt referanse til gjeldende skattemelding. 

Cellen nedenfor henter ut gjeldende dokumentrefranse printer ut responsen fra hent gjeldende kallet 

In [8]:
sjekk_svar = r_gjeldende

sme_og_naering_respons = xmltodict.parse(sjekk_svar.text)
skattemelding_base64 = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]["skattemeldingdokument"]
sme_base64 = skattemelding_base64["content"]
dokref = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]['skattemeldingdokument']['id']
decoded_sme_xml = decode_dokument(skattemelding_base64)
sme_utkast = xml.dom.minidom.parseString(decoded_sme_xml["content"]).toprettyxml()
sme_dict = xmltodict.parse(decoded_sme_xml['content'])
partsnummer = sme_dict['skattemelding']['partsnummer']

print(f"Responsen fra hent gjeldende ser slik ut, gjeldende dokumentrerefanse er {dokref}")
print(sjekk_svar.request.method ,sjekk_svar.request.url)
print(xml.dom.minidom.parseString(sjekk_svar.text).toprettyxml())


Responsen fra hent gjeldende ser slik ut, gjeldende dokumentrerefanse er SKI:755:151244
GET https://idporten-api-sbstest.sits.no/api/skattemelding/v2/2022/312787016
<?xml version="1.0" ?>
<skattemeldingOgNaeringsspesifikasjonforespoerselResponse xmlns="no:skatteetaten:fastsetting:formueinntekt:skattemeldingognaeringsspesifikasjon:forespoersel:response:v2">
	<dokumenter>
		<skattemeldingdokument>
			<id>SKI:755:151244</id>
			<encoding>utf-8</encoding>
			<content>PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiPz48c2thdHRlbWVsZGluZyB4bWxucz0idXJuOm5vOnNrYXR0ZWV0YXRlbjpmYXN0c2V0dGluZzpmb3JtdWVpbm50ZWt0OnNrYXR0ZW1lbGRpbmc6dXBlcnNvbmxpZzpla3N0ZXJuOnYyIj48cGFydHNudW1tZXI+MzAwMTU4OTMyMzwvcGFydHNudW1tZXI+PGlubnRla3RzYWFyPjIwMjI8L2lubnRla3RzYWFyPjwvc2thdHRlbWVsZGluZz4=</content>
			<type>skattemeldingUpersonligUtkast</type>
		</skattemeldingdokument>
	</dokumenter>
</skattemeldingOgNaeringsspesifikasjonforespoerselResponse>


In [15]:
#Legg merge til dokumenter.dokument.type = skattemeldingUpersonlig

with open(f"../../../src/resources/eksempler/{innteksår}/Naeringspesifikasjon-AS-v3.xml", 'r') as f:
    naering_as_xml = f.read()
    
innsendingstype = "komplett"
naeringsspesifikasjoner_as_b64 = base64.b64encode(naering_as_xml.encode("utf-8"))
naeringsspesifikasjoner_as_b64 = str(naeringsspesifikasjoner_as_b64.decode("utf-8"))
naeringsspesifikasjoner_base64=naeringsspesifikasjoner_as_b64


with open(f"../../../src/resources/eksempler/{innteksår}/upersonligSkattemeldingV2.xml", 'r') as f:
    sme_verdi_bak_aksjer = f.read().format(partsnummer=partsnummer)
    sme_b64 = base64.b64encode(sme_verdi_bak_aksjer.encode("utf-8"))
    sme_b64 = str(sme_b64.decode("utf-8"))


valider_konvlutt_v2 = """
<?xml version="1.0" encoding="utf-8" ?>
<skattemeldingOgNaeringsspesifikasjonRequest xmlns="no:skatteetaten:fastsetting:formueinntekt:skattemeldingognaeringsspesifikasjon:request:v2">
    <dokumenter>
        <dokument>
            <type>skattemeldingUpersonlig</type>
            <encoding>utf-8</encoding>
            <content>{sme_base64}</content>
        </dokument>
        <dokument>
            <type>naeringsspesifikasjon</type>
            <encoding>utf-8</encoding>
            <content>{naeringsspeifikasjon_base64}</content>
        </dokument>
    </dokumenter>
    <dokumentreferanseTilGjeldendeDokument>
        <dokumenttype>skattemeldingUpersonlig</dokumenttype>
        <dokumentidentifikator>{dok_ref}</dokumentidentifikator>
    </dokumentreferanseTilGjeldendeDokument>
    <inntektsaar>{innteksår}</inntektsaar>
    <innsendingsinformasjon>
        <innsendingstype>{innsendingstype}</innsendingstype>
        <opprettetAv>TurboSkatt</opprettetAv>
    </innsendingsinformasjon>
</skattemeldingOgNaeringsspesifikasjonRequest>
""".replace("\n","")


naering_as = valider_konvlutt_v2.format(sme_base64=sme_b64,
                                         naeringsspeifikasjon_base64=naeringsspesifikasjoner_base64,
                                         dok_ref=dokref,
                                         innteksår=innteksår,
                                         innsendingstype=innsendingstype)

# Valider utkast sme med næringsopplysninger

In [17]:
valider_respons = valider(naering_as, inntektsår=innteksår, s=s, tin=orgnr_as, idporten_header=idporten_header)
resultatAvValidering = xmltodict.parse(valider_respons.text)["skattemeldingOgNaeringsspesifikasjonResponse"]["resultatAvValidering"]

if valider_respons:
    print(valider_respons.status_code, valider_respons.headers, valider_respons.text)
    print(resultatAvValidering + "\n")
    print(xml.dom.minidom.parseString(valider_respons.text).toprettyxml())
else:
    print(valider_respons.status_code, valider_respons.headers, valider_respons.text)

200 {'korrelasjonsid': 'ca27ebb4-c2a0-1fff-9436-58aa16bce247', 'content-type': 'application/xml;charset=UTF-8', 'content-length': '631', 'date': 'Sun, 02 Oct 2022 18:30:41 GMT', 'Expires': 'Thu, 01 Dec 1994 16:00:00 GMT', 'Strict-Transport-Security': 'max-age=16070400; includeSubDomains'} <skattemeldingOgNaeringsspesifikasjonResponse xmlns="no:skatteetaten:fastsetting:formueinntekt:skattemeldingognaeringsspesifikasjon:response:v2"><avvikVedValidering><avvik><avvikstype>xmlValideringsfeilPaaSkattemelding</avvikstype></avvik><avvik><avvikstype>xmlValideringsfeilPaaNaeringsopplysningene</avvikstype></avvik></avvikVedValidering><resultatAvValidering>validertMedFeil</resultatAvValidering><aarsakTilValidertMedFeil>xmlValideringsfeilPaaSkattemelding</aarsakTilValidertMedFeil><aarsakTilValidertMedFeil>xmlValideringsfeilPaaNaeringsopplysningene</aarsakTilValidertMedFeil></skattemeldingOgNaeringsspesifikasjonResponse>
validertMedFeil

<?xml version="1.0" ?>
<skattemeldingOgNaeringsspesifikasjonR

# Altinn 3

1. Hent Altinn Token
2. Oppretter en ny instans av skjemaet
3. Laster opp skattemeldingen og næringsopplysninger som et vedlegg

In [ ]:
#1
altinn3_applikasjon = "skd/formueinntekt-skattemelding-v2"
altinn_header = hent_altinn_token(idporten_header)
#2
tesmiljø = None #Default testmiljø skal brukes for SBS innsending
instans_data = opprett_ny_instans_med_inntektsaar(altinn_header, innteksår, tesmiljø, orgnr=orgnr_as, appnavn=altinn3_applikasjon)

## Last opp skattemelding

In [ ]:
#Last opp skattemeldingen
req_send_inn = last_opp_skattedata(instans_data, altinn_header, 
                                   xml=naering_as,
                                   data_type="skattemeldingOgNaeringsspesifikasjon",
                                   appnavn=altinn3_applikasjon)
req_send_inn

### Sett statusen klar til bekreftelse.

In [ ]:
req_bekreftelse = endre_prosess_status(instans_data, altinn_header, "next", appnavn=altinn3_applikasjon)

### Åpner skattemelding visningsklient for å se beregnet skattemelding med næringsspesifikasjon.

In [ ]:
from skatteetaten_api import skattemelding_visning

url_skattemelding_visning = skattemelding_visning(instans_data, appnavn=altinn3_applikasjon)
print(url_skattemelding_visning)

### Sett statusen klar til henting av skatteetaten.

In [ ]:
req_bekreftelse = endre_prosess_status(instans_data, altinn_header, "next", appnavn=altinn3_applikasjon)
req_bekreftelse